# Assignment 2 for Group44

In [1]:
from typing import List, Tuple
import heapq
from collections import deque

def heuristic(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_directions() -> List[Tuple[int, int]]:
    return [(0, 1), (0, -1), (1, 0), (-1, 0)]

def neighbors(grid: List[List[str]], node: Tuple[int, int]) -> List[Tuple[int, int]]:
    directions = get_directions()
    neighbors = []
    for dx, dy in directions:
        nx, ny = node[0] + dx, node[1] + dy
        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] != "@" and grid[nx][ny] != "$":
            neighbors.append((nx, ny))
    return neighbors

def bfs(grid: List[List[str]], start: Tuple[int, int], end: Tuple[int, int]) -> List[Tuple[int, int]]:
    queue = deque([start])
    visited = {start}
    came_from = {start: None}
    while queue:
        node = queue.popleft()
        if node == end:
            path = []
            while node is not None:
                path.append(node)
                node = came_from[node]
            return path[::-1]
        for neighbor in neighbors(grid, node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                came_from[neighbor] = node
    return []

def solve(grid: List[List[str]]) -> int:
    start, item = None, None
    targets = []
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == "R":
                start = (i, j)
            elif grid[i][j] == "$":
                item = (i, j)
            elif grid[i][j] == "T":
                target = (i, j)
                targets.append(target)
    
    # Invalid grid
    if not start or len(target)==0 or not item:
        return -2

    # Find all possible positions from where the robot can push the item towards the target
    possible_positions = []
    for dx, dy in get_directions():
        nx, ny = item[0] + dx, item[1] + dy
        opposite = item[0] - dx, item[1] - dy
        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and 0 <= opposite[0] < len(grid) and 0 <= opposite[1] < len(grid[0]) and grid[opposite[0]][opposite[1]] != "@":
            bfs_path = bfs(grid, start, (nx, ny))
            if bfs_path:
                possible_positions.append((opposite, bfs_path))

    # A* search from item to target
    min_distance_lists = []
    for target in targets:
        min_distance = float('inf')
        path_to_target = None
        print(f"Target {target}:")
        
        for start, bfs_path in possible_positions:
            print("Path from robot to item: ", bfs_path)
            queue = [(heuristic(start, target), 0, start, None)]
            visited = {start: 0}
            came_from = {start: None}
            while queue:
                _, g, node, _ = heapq.heappop(queue)
                if node == target:
                    if g < min_distance:
                        min_distance = g
                        path_to_target = []
                        while node is not None:
                            path_to_target.append(node)
                            node = came_from[node]
                    break
                for neighbor in neighbors(grid, node):
                    new_cost = g + 1
                    if neighbor not in visited or new_cost < visited[neighbor]:
                        visited[neighbor] = new_cost
                        f = new_cost + heuristic(neighbor, target)
                        heapq.heappush(queue, (f, new_cost, neighbor, node))
                        came_from[neighbor] = node

        if path_to_target is None:
            min_distance_lists.append(-1)
            print("Path from item to target: None")
            print(f"Minimum: -1")
        else:
            min_distance_lists.append(len(path_to_target[::-1]))
            print("Path from item to target: ", path_to_target[::-1])
            print(f"Minimum: {len(path_to_target[::-1])}")

    print(f"Overall minimum: {min(min_distance_lists)}")
    return min(min_distance_lists)


In [2]:
# test case 1
grid1 = [["@", "@", "@", "@", "@", "@"],
         ["@", "@", "@", "@", "T", "@"],
         ["@", "#", "$", "#", "#", "@"],
         ["@", "#", "@", "@", "#", "@"],
         ["@", "R", "#", "#", "#", "@"],
         ["@", "T", "@", "@", "@", "@"]]
answer1 = 3
result1 = solve(grid1)
assert result1 == answer1, f"Test case 1: expected {answer1}, got {result1}"
print('Passed test case 1...')

Target (1, 4):
Path from robot to item:  [(4, 1), (4, 2), (4, 3), (4, 4), (3, 4), (2, 4), (2, 3)]
Path from robot to item:  [(4, 1), (3, 1), (2, 1)]
Path from item to target:  [(2, 3), (2, 4), (1, 4)]
Minimum: 3
Target (5, 1):
Path from robot to item:  [(4, 1), (4, 2), (4, 3), (4, 4), (3, 4), (2, 4), (2, 3)]
Path from robot to item:  [(4, 1), (3, 1), (2, 1)]
Path from item to target:  [(2, 1), (3, 1), (4, 1), (5, 1)]
Minimum: 4
Overall minimum: 3
Passed test case 1...


In [3]:
# test case 2
grid2 = [["@", "T", "@", "@", "@", "@"],
         ["@", "#", "@", "@", "@", "@"],
         ["@", "#", "#", "#", "$", "@"],
         ["@", "#", "@", "@", "@", "@"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "@", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid2)
assert result2 == answer2, f"Test case 2: expected {answer2}, got {result2}"
print('Passed test case 2...')

Target (0, 1):
Path from item to target: None
Minimum: -1
Target (4, 4):
Path from item to target: None
Minimum: -1
Overall minimum: -1
Passed test case 2...


In [4]:
# test case 3: blocked by edges
grid3 = [["@", "T", "@", "@", "@", "@"],
         ["@", "#", "@", "@", "@", "#"],
         ["@", "#", "#", "#", "#", "$"],
         ["@", "#", "@", "@", "@", "#"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "@", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid3)
assert result2 == answer2, f"Test case 3: expected {answer2}, got {result2}"
print('Passed test case 3...')

Target (0, 1):
Path from item to target: None
Minimum: -1
Target (4, 4):
Path from item to target: None
Minimum: -1
Overall minimum: -1
Passed test case 3...


In [5]:
# test case 4: blocked by edges
grid3 = [["$", "T", "@", "@", "@", "@"],
         ["#", "#", "@", "@", "@", "#"],
         ["@", "#", "#", "#", "#", "#"],
         ["@", "#", "@", "@", "@", "#"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "@", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid3)
assert result2 == answer2, f"Test case 4: expected {answer2}, got {result2}"
print('Passed test case 4...')

Target (0, 1):
Path from item to target: None
Minimum: -1
Target (4, 4):
Path from item to target: None
Minimum: -1
Overall minimum: -1
Passed test case 4...


In [6]:
# test case 5: robot blocked by obstacles
grid3 = [["#", "T", "@", "@", "@", "@"],
         ["#", "$", "@", "@", "@", "#"],
         ["@", "#", "#", "#", "#", "#"],
         ["@", "@", "@", "@", "@", "#"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "@", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid3)
assert result2 == answer2, f"Test case 5: expected {answer2}, got {result2}"
print('Passed test case 5...')

Target (0, 1):
Path from item to target: None
Minimum: -1
Target (4, 4):
Path from item to target: None
Minimum: -1
Overall minimum: -1
Passed test case 5...


In [7]:
# test case 6: robot blocked by obstacles
grid3 = [["#", "T", "@", "@", "@", "@"],
         ["#", "$", "@", "@", "@", "#"],
         ["@", "#", "#", "#", "#", "#"],
         ["@", "@", "@", "@", "@", "#"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "#", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid3)
assert result2 == answer2, f"Test case 6: expected {answer2}, got {result2}"
print('Passed test case 6...')

Target (0, 1):
Path from item to target: None
Minimum: -1
Target (4, 4):
Path from item to target: None
Minimum: -1
Overall minimum: -1
Passed test case 6...


In [ ]:
# test case 7: robot blocked by obstacles
grid3 = [["#", "T", "@", "@", "@", "@"],
         ["#", "$", "@", "@", "@", "#"],
         ["@", "#", "#", "#", "#", "#"],
         ["@", "@", "@", "@", "@", "#"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "#", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid3)
assert result2 == answer2, f"Test case 6: expected {answer2}, got {result2}"
print('Passed test case 6...')